Uczenie DCGAN na zbiorze MNIST (z użyciem Keras)
Użyjemy sieci CNN
Zbudujemy dwie sieci: Generator i Dyskryminator, gdzie jedna będzie tworzyć obrazy podobne do tych ze zbioru danych
a druga sieć będzie weryfikować obrazy oryginalne i wygenerowane
Na pewnym etapie sieć dyskryminująca podpowiada generatorowi jak tworzyć najlepsze podróbki obrazów!!!

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
from tensorflow.keras.layers import Dense, Input, Activation
from tensorflow.keras.layers import Conv2D, Flatten
from tensorflow.keras.layers import Reshape, Conv2DTranspose
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.datasets import mnist

In [3]:
import numpy as np
import os
import matplotlib.pyplot as plt
import math
import argparse

In [4]:
#funkcja budująca sieć generator
def build_generator(inputs,image_size):
    """
    Stos warstw BN-Relu-Conv2DTranspose do generowania fałszywych obrazów.
    wyjściowa funkcja aktywacji: sigmoid /ta fukcja łatwiej osiąga zbieżność niż tanh/
    ARGUMENTY:
    inputs -> warstwa wejściowa - wektor z
    image_size (tensor) -> zakładamy kształ kwadratu
    ZWRACA:
    generator -> model generatora
    """

    #parametry sieci
    image_resize = image_size//4
    kernel_size = 5
    layer_filters = [128,64,32,1]

    x = Dense((image_resize**2)*layer_filters[0])(inputs)
    x = Reshape((image_resize,image_resize,layer_filters[0]))(x)

    for filters in layer_filters:
        if filters>layer_filters[-2]:
            strides = 2
        else:
            strides = 1
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2DTranspose(filters=filters,
                            kernel_size=kernel_size,
                            strides=strides,
                            padding='same')(x)
    x = Activation('sigmoid')(x)
    generator = Model(inputs,x,name='generator')
    return generator

In [5]:
#konstruowanie modelu dyskryminatora
def build_discriminator(inputs):
    """
    Stos warstw LeakyRelLU-Conv2D do odróżniania obrazów prawdziwych.
    Rezygnacja z BN -> brak osiągnięcia zbieżności
    ARGUMENTY:
    inputs -> warstwa wejściowa dyskryminatora
    ZWRACA:
    dyskryminator -> model dyskryminatora
    """
    kernel_size = 5
    layer_filters = [32,64,128,256]

    x = inputs
    for filters in layer_filters:
        if filters == layer_filters[-1]:
            strides = 1
        else:
            strides = 2
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(filters=filters,
                   kernel_size=kernel_size,
                   strides=strides,
                   padding='same')(x)
    x = Flatten()(x)
    x = Dense(1)(x)
    x = Activation('sigmoid')(x)
    discriminator = Model(inputs,x,name='discriminator')
    return discriminator